In [ ]:
import json
with open("outputs/buckets07-22_16:37.json","r") as file:
  buckets = json.load(file)
bucket_map={action:bucket[0] for bucket in buckets for action in bucket}

In [ ]:
import pandas as pd
bucket_names = [bucket[0] for bucket in buckets if type(bucket)==list and len(bucket)>0]
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df=pd.concat([pd.read_csv(dataset) for dataset in datasets])
history_df["bucket"]=history_df["action"].apply(lambda action:bucket_map[action])
llm_buckets = []
# with open("./outputs/bucket_names_output.jsonl", "r") as file:
#   for line in file:
#     if line.strip():  # Skip empty lines
#       response = json.loads(line)
#       llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
# history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))

# Add next bill_id column to compare
history_df["next_bill_id"]=history_df["bill_id"].shift(-1)
history_df["next_bucket"]=history_df["bucket"].shift(-1)
history_df["next_bucket"]=history_df.apply(lambda row:row["next_bucket"] if row["next_bill_id"]==row["bill_id"] else None,1)
# history_df["next_bucket_llm"]=history_df["llm_bucket"].shift(-1)
# history_df["next_llm_bucket"]=history_df.apply(lambda row:row["next_bucket_llm"] if row["next_bill_id"]==row["bill_id"] else None,1)





# display(history_df)
# for i,row in history_df.iterrows():
#   print(row.bucket)
# for action in history_df:
#   history_df

In [ ]:
display(history_df)

In [ ]:
# Constructing the graph
import networkx as nx
g_dict={}
for bucket1 in bucket_names:
  g_dict[bucket1]={}
  for bucket2 in bucket_names:
    # print(bucket1,bucket2)
    g_dict[bucket1][bucket2]=0
  g_dict[bucket1][None]=0
for i, row in history_df.iterrows():
  # print(row["bucket"])
  # print(row["next_bucket"])
  g_dict[row["bucket"]][row["next_bucket"]]+=1
g = nx.DiGraph()
for bucket1 in bucket_names:
  for bucket2 in bucket_names:
    g.add_edge(bucket1,bucket2,weight=g_dict[bucket1][bucket2])
display(g)
del g_dict
# for bucket1 in bucket_names:
#   for bucket2 in bucket_names:
#     g_dict[bucket1][bucket2]+=1
# for bucket1 in bucket_names:
#   for bucket2 in bucket_names:
#     g.
# g_tuples = {bucket:g_dict[bucket].items() for bucket in bucket_names}
# display(g_tuples)

In [ ]:
print(g.number_of_nodes())

In [ ]:
import networkx as nx
print(nx.diameter(g))
# nx.draw(g)
